# Preprocesado de datos

El preprocesado de datos es una fase indispensable para el correndo aprendizaje por partes de los algoritmos de Deeplearning. Se ha demostrado empíricamente que una correcta preparación y normalización de los datos permiten hallar soluciones más cercanas a la optima que con datos no procesados.

Es importante tener en cuenta que no existe una metodología de preprocesado única, y que es necesario adaptarse al tipo de dato que estamos tratando. Para este proyecto, además, existe una dificultad adicional, y es la existencia de diferentes procedencias para los datos, pues en total se dispone de 5 datasets distintos, cada uno recopilado con diferentes metodologías e instrumentación. Por tanto, será clave adaptarse a cada uno de los destinos, y realizar la partición final de forma estratificada para evitar sesgos que perturben el resutado.

Más adelante, profundizaremos en este aspecto, pero en primer lugar, debemos leer cada uno de los conjuntos de datos disponibles, y examinar de cuántos elementos disponemos en cada uno, para establecer la proporción de entrenamiento-test oportuna.

In [1]:
# Librerías utilizadas por el script
import os
import cv2
import zipfile
import csv
import pathlib
import shutil

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset disponibles

Haciendo uso de los recursos disponibles públicamente, se han tomado los siguientes datasets para realizar el experimento:
- ISIC: es el mayor conjuntos de datos cutáneos disponible en abierto, y contiene imágenes de todo tipo de pieles y procedencias, pero con especial énfasis en las personas de origen europeo y americano.
- ASAN: Conjunto de datos provenientes del hospital con este mismo nombre, con lesiones en personas de origen asiático.
- PAD UFES 20: conjunto de datos de lesiones variadas de pacientes latinoamericanos.
- PH2: banco de datos de pacientes brasileños con lesiones potencialmente cancerosas.
- Severance: base de datos con lesiones cutáneas en población asiática, con contenido tanto benigno como cancerígeno.

Para unificar la notación de los datos, se creará un código para la notación de cada una de las clases que permitan un procesamiento común de todos los datos sin depender del origen de este. Para ello, se creará un nuevo archivo .csv donde se anotará el path de la imagen, su clase asociada, y el tipo general de la misma (beningna o maligna). La metainformación asociada, de momento, quedará relegado a un segundo plano hasta el estudio estadístico de los datos.

In [2]:
# Directorios de cada dataset

ISIC_PATH = "datasets/ISIC"
ASAN_PATH = "datasets/Asan"
PAD_UFES_PATH = "datasets/PAD_UFES_20"
PH2_PATH = "datasets/PH2/PH2Dataset"
SEVERANCE = "datasets/Severance"

In [3]:
# Funciones comunes para la lectura y muestra de datos

'''
This function receives a string with the filename of the image to read,
and a flag indicating if we want to read it in color/RGB (flagColor=1) or gray level (flagColor=0)

Example of use:
im1=readIm(get_image('apple.jpg'),0)
'''


def readIm(filename, flagColor=1):
    # cv2 reads BGR format
    im = cv2.imread(filename)
    # change to  RGB and return the image
    if (flagColor):
        return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # change from BGR to grayscale instead if flag is 0
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)


'''
This function receives an array of arbitrary real numbers (that could include even negative values),
and returns an 'image' in the range [0,1].
flag_GLOBAL allows the user to normalize the whole image (including all channels) or to normalize
each channel/band independently.
'''


def rangeDisplay01(im, flag_GLOBAL=True):
    im = im.astype(float)
    if flag_GLOBAL:
        im = (im - im.min()) / (im.max() - im.min())
    else:
        # bands normalization
        for band in range(im.shape[2]):
            im[:, :, band] = (im[:, :, band] - im[:, :, band].min()) / (im[:, :, band].max() - im[:, :, band].min())
            # Note: remember that, for plt.imshow with RGB data, the valid range is [0..1] for floats and [0..255] for integers.
    return im


"""
Función para mostrar imágenes en pantalla en color y blanco negro. Permite realizar
aumento sobre las mismas para apreciar un mayor detalle.

Entrada:
    im: imagen leída en formato ndarray
    title: nombre que recibe el marco en pantalla
    factor: factor de aumento de la image, "zoom"
"""


def displayIm(im, title='Result', factor=2):
    # First normalize range
    max = np.max(im)
    min = np.min(im)
    if min < 0 or max > 255:
        im = rangeDisplay01(im, flag_GLOBAL=True)
    if len(im.shape) == 3:
        # im es tribanda
        plt.imshow(im, cmap='jet')
    else:
        # im es monobanda
        plt.imshow(im, cmap='gray')
    figure_size = plt.gcf().get_size_inches()
    plt.gcf().set_size_inches(factor * figure_size)
    plt.title(title)
    plt.xticks([]), plt.yticks([])  # eliminamos numeración
    plt.show()

In [4]:
# En estas variables, se acumularán las clases de cada dataset para matener notación común
global_y = set()

### ISIC Skin Dataset

Se trata del dataset de mayor tamaño del conjunto. Contiene 31 clases identificadas, tanto de lesiones benignas y malignas de la piel. En total, se dispone de 53738, las cuales ya han sido filtradas para asegurarse de que no exista redundancia por las herramientas online de la galería ISIC: https://gallery.isic-archive.com/

In [5]:
def extractISIC(path: str):
    # Obtener una lista de todos los archivos ZIP en la carpeta especificada
    archivos_zip = [f for f in os.listdir(path) if f.lower().endswith('.zip')]

    # Iterar sobre cada archivo ZIP
    for archivo_zip in archivos_zip:
        ruta_archivo_zip = os.path.join(path, archivo_zip)
        carpeta_salida = os.path.splitext(ruta_archivo_zip)[0]  # Eliminar la extensión .zip

        # Comprobar si la carpeta de salida ya existe
        if not os.path.exists(carpeta_salida):
            os.makedirs(carpeta_salida)  # Crear la carpeta de salida

            # Extraer el contenido del archivo ZIP en la carpeta de salida
            with zipfile.ZipFile(ruta_archivo_zip, 'r') as zip_ref:
                zip_ref.extractall(carpeta_salida)
            print(f"Extraído {archivo_zip} en {carpeta_salida}")
        else:
            print(f"Omitido {archivo_zip}. {carpeta_salida} ya existe.")


def listar_clases(path):
    # Obtener una lista de todas las carpetas en el path
    return [nombre for nombre in os.listdir(path) if os.path.isdir(os.path.join(path, nombre))]


def definir_etiquetas(path):
    clases = listar_clases(path)
    print(clases)
    return [clase.replace(" ", "_").lower() for clase in clases]


def crear_csv(path, clases, nombre_dataset):
    # Inicializa una lista vacía para almacenar la información de los archivos
    info_archivos = []
    i = 0

    # Itera sobre cada carpeta en la carpeta raíz
    for nombre_carpeta in os.listdir(path):
        ruta_carpeta = os.path.join(path, nombre_carpeta)

        if os.path.isdir(ruta_carpeta):

            # Itera sobre cada archivo en la carpeta
            for nombre_archivo in os.listdir(ruta_carpeta):
                ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
                if os.path.isfile(
                        ruta_archivo) and nombre_archivo != "metadata.csv" and nombre_archivo != "attribution.txt":
                    # Agrega la información del archivo a la lista
                    info_archivos.append((nombre_archivo, ruta_archivo, clases[i], nombre_dataset))
            i += 1

    # Define la ruta del archivo CSV
    ruta_archivo_csv = "preprocessedData.csv"

    # Escribe la información de los archivos en el archivo CSV
    with open(ruta_archivo_csv, "w", newline="") as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(["image", "dir", "class", "dataset"])  # Escribe la cabecera
        escritor_csv.writerows(info_archivos)  # Escribe la información de los archivos

In [6]:

# Accedemos al directorio de ISIC
print(os.getcwd())

# Extraemos cada zip, en caso de que no exista
extractISIC(ISIC_PATH)

# Tomamos los nombres de cada imagen, y le asociamos su etiqueta manualmente
print(os.getcwd())

isic_y = definir_etiquetas(ISIC_PATH)
global_y = global_y.union(set(isic_y))

# Creamos CSV
crear_csv(ISIC_PATH, isic_y, "ISIC")
print(os.getcwd())

C:\Users\Cris1\Documents\TFG
Extraído Acrochordon.zip en datasets/ISIC\Acrochordon
Extraído Acticnic keratosis.zip en datasets/ISIC\Acticnic keratosis
Extraído AIMP.zip en datasets/ISIC\AIMP
Extraído Angiofibroma or fibreus papule.zip en datasets/ISIC\Angiofibroma or fibreus papule
Extraído Angiokeratoma.zip en datasets/ISIC\Angiokeratoma
Extraído Angioma.zip en datasets/ISIC\Angioma
Extraído Atypical melanocytic proliferation.zip en datasets/ISIC\Atypical melanocytic proliferation
Extraído Atypical spitz tumor.zip en datasets/ISIC\Atypical spitz tumor
Extraído Basal Cell Carcinoma.zip en datasets/ISIC\Basal Cell Carcinoma
Extraído Cafe au lait macule.zip en datasets/ISIC\Cafe au lait macule
Extraído Clear cell acarthoma.zip en datasets/ISIC\Clear cell acarthoma
Extraído Dermatofibroma.zip en datasets/ISIC\Dermatofibroma
Extraído Lentigo NOS.zip en datasets/ISIC\Lentigo NOS
Extraído Lentigo Simplex.zip en datasets/ISIC\Lentigo Simplex
Extraído Lichenoid keratosis.zip en datasets/ISIC\L

### ASAN Dataset

Este dataset es el segundo de mayor tamaño recogido. En total, dispone de 17,125 imágenes de 12 clases distintas. Muchas de estas clases tratan la misma enfermedad, pero distinguen si se ha realizado biopsia o no para verificarlo (aunque todos los resultados han sido confirmados posteriormente tras estudiar su evolución).

La dificultad de este conjunto de datos se debe al formato de almacenaje escogido: todas las imágenes fueron guardadas en estructura de rejilla, provocando la existencia de cientos de imágenes en un mismo espacio separado por bordes blancos. Por tanto, habrá que realizar una etapa de preprocesado más profundo que ISIC para dividir correctamente la imagen y remover los bordes blancos para evitar sesgos en los resultados del modelo final.

El código se divide en dos fases: una primera fase de separación, y la segunda de barajado, donde uniremos el conjunto de entrenamiento y test dividido anteriormente, ya que la proporción elegida para test fue de un apenas 10%, y no se conoce el grado de aleatoriedad del criterio de seperación elegido.

In [7]:
grosor_borde = 8  # Constante del borde a eliminar (px)


# Recortado
def recortarImagenesASAN(path, i, title="ASAN"):
    os.chdir(path)

    files = [f for f in pathlib.Path().iterdir() if f.is_file()][1:]

    names = []
    diss_class = []

    for f in files:
        name = str(f)[str(f).rfind("#") + 1:-4]

        if ".png" in str(f):
            print("Procesando ", name)
            if not os.path.exists(name):
                os.makedirs(name)

            image = cv2.imread(str(f), cv2.IMREAD_UNCHANGED)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)

            contours, _ = cv2.findContours(gradient, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                (x, y, w, h) = cv2.boundingRect(cnt)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 0))
                box_image = image[y: y + h, x: x + w]

                # Recorta la imagen para eliminar el borde
                img_sin_borde = box_image[grosor_borde:-grosor_borde, grosor_borde:-grosor_borde]
                cv2.imwrite(f"{name}/{title}_{i}.png", img_sin_borde)

                names.append(f"{title}_{i}.png")
                diss_class.append(name)

                i += 1

    os.chdir("../../../")

    return i

Como el código para crear el fichero CSV ya ha sido creado, sólo debemos añadir las nuevas imágenes al fichero respetando ese formato. 

Para las etiquetas, al tratarse de las mismas enfermedades ya presentes en el dataset de ISIC, debemos asegurarnos que éstas tengan el mismo formato; si cometemos algún error en su escritura, podríamos provocar que el modelo interprete la misma enfermedad como dos distintas.

In [8]:
def aniadir_csv(path, clases, nombre_dataset):
    # Inicializa una lista vacía para almacenar la información de los archivos
    info_archivos = []
    i = 0

    # Itera sobre cada carpeta en la carpeta raíz
    for nombre_carpeta in os.listdir(path):
        ruta_carpeta = os.path.join(path, nombre_carpeta)

        if os.path.isdir(ruta_carpeta):

            # Itera sobre cada archivo en la carpeta
            for nombre_archivo in os.listdir(ruta_carpeta):
                ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
                if os.path.isfile(
                        ruta_archivo) and nombre_archivo != "metadata.csv" and nombre_archivo != "attribution.txt":
                    # Agrega la información del archivo a la lista
                    info_archivos.append((nombre_archivo, ruta_archivo, clases[i], nombre_dataset))
                    print(info_archivos[i])
            i += 1

    # Define la ruta del archivo CSV
    ruta_archivo_csv = "preprocessedData.csv"

    # Escribe la información de los archivos en el archivo CSV

    with open(ruta_archivo_csv, "a", newline="") as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerows(info_archivos)  # Escribe la información de los archivos

Una vez definida la función de adición, podemos proceder a la escritura de las nuevas imágenes y su directorio. Para homogeneizar volver a crear las particiones de train y test, se ha procedido a su merge.

In [9]:
# Ejecutamos la separación del conjunto de train

i = 0  # identificador de imagen

print(os.getcwd())

i = recortarImagenesASAN(ASAN_PATH + "/train_dataset", i)
i = recortarImagenesASAN(ASAN_PATH + "/test_dataset", i)

os.mkdir(ASAN_PATH + "/dataset")

# Fusionamos ambas carpetas
shutil.copytree(ASAN_PATH + "/train_dataset", ASAN_PATH + "/dataset", dirs_exist_ok=True)
shutil.copytree(ASAN_PATH + "/test_dataset", ASAN_PATH + "/dataset", dirs_exist_ok=True)

# Añádimos nuevas etiquetas
asan_y = definir_etiquetas(ASAN_PATH + "/dataset")
global_y = global_y.union(set(asan_y))

aniadir_csv(ASAN_PATH + "/dataset", asan_y, "ASAN")


C:\Users\Cris1\Documents\TFG
Procesando  actinic keratosis
Procesando  basal cell carcinoma
Procesando  dermatofibroma
Procesando  hemangioma
Procesando  intraepithelial carcinoma
Procesando  lentigo nos
Procesando  melanoma
Procesando  nevus
Procesando  pyogenic granuloma
Procesando  Seborreic Keratosis
Procesando  Squamous cell carcinoma
Procesando  wart
Procesando  actinic keratosis
Procesando  dermatofibroma
Procesando  hemangioma
Procesando  lentigo nos
Procesando  nevus
Procesando  pyogenic granuloma
Procesando  Seborreic Keratosis
Procesando  wart
Procesando  acticnic keratosis
Procesando  basal cell carcinoma
Procesando  dermatofibroma
Procesando  hemangioma
Procesando  intraepithelial carcinoma
Procesando  lentigo nos
Procesando  melanoma
Procesando  nevus
Procesando  pyogenic granuloma
Procesando  Seborreic Keratosis
Procesando  Squamous cell carcinoma
Procesando  wart
['acticnic keratosis', 'actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'hemangioma', 'intraep

Podemos comprobar en la salida de que todas las clases se han tratado correctamente.

#### Añadido: dataset Hallym

Dentro del fichero que proporcionaba el dataset ASAN, podemos encontrar de forma adicional un pequeño dataset con algunas imágenes extras. Estas pertenecen al dataset de Hallym, el cual recopila sobre 150 imágenes de carcinoma de células basales ubicados en la espalda de diversos pacientes. Para aprovechar este conjunto de datos, podemos operarar de la misma forma que con ASAN, ya que el formato de presentación es el mismo.

Debemos prestar especial atención a los nombres de las clases, ya que en este caso aparecen como "bcc", para denotar la enfermedad. Debemos renombrar la clase a "basal_cell_carcinoma".

In [10]:
i = 0  # identificador de imagen

i = recortarImagenesASAN(ASAN_PATH + "/Hallym_dataset", i, "HALLYM")

# añadimos a nueva carpeta "basal_cell_carcinoma"
# comprobamos primero si    existe para crearla


# movemos las imágenes y borramos las carpetas anteriores
shutil.copytree(ASAN_PATH + "/Hallym_dataset/back bcc", ASAN_PATH + "/Hallym_dataset/basal_cell_carcinoma",
                dirs_exist_ok=True)
shutil.copytree(ASAN_PATH + "/Hallym_dataset/hallym bcc", ASAN_PATH + "/Hallym_dataset/basal_cell_carcinoma",
                dirs_exist_ok=True)

shutil.rmtree(ASAN_PATH + "/Hallym_dataset/back bcc")
shutil.rmtree(ASAN_PATH + "/Hallym_dataset/hallym bcc")

hallym_y = definir_etiquetas(ASAN_PATH + "/Hallym_dataset")

#global_y = global_y.union(set(hallym_y))

aniadir_csv(ASAN_PATH + "/Hallym_dataset/", hallym_y, "HALLYM")

Procesando  back bcc
Procesando  hallym bcc
['basal_cell_carcinoma']
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma', 'HALLYM')
('HALLYM_0.png', 'datasets/Asan/Hallym_dataset/basal_cell_carcinoma\\HALLYM_0.png', 'basal_cell_carcinoma

### PAD UFES 20

Se trata de un dataset diseñado para el entramiento de sistemas de asistencia en diagnóstico computado, donde el experto dermatólogo puede utilizarlo como un medio de apoyo. Contiene 6 enfermedades distintas, siendo 3 cancerosas (células basales, células escamosas o melanoma maligno) y 3 benignas (actinic keratosis, nevus, keratosis seborreica).

Todos los casos cancerosos han sido probados mediante biopsia, por lo que tenemos total certeza de los resultados recopilados. A su vez, también contenemos gran cantidad de metados asociados a la edad, antencedentes familiares, hábitos, entre otros.

#### Estructura de los datos
La organización de las imágenes es más sencilla que ASAN, pues las imágenes son individuales. Además, dispone de un fichero CSV adecuadamente organizado. La única modificación necesaria es actualizar el path de cada imagen y la nomenclatura del diagnóstico de la enfermedad por la ya utilizada.

In [11]:
# Lectura del csv dado
df_pad_ufes = pd.read_csv(PAD_UFES_PATH + '/metadata.csv')
display(df_pad_ufes)

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


In [12]:
# Modificamos las etiquetas
df_pad_ufes['diagnostic'] = df_pad_ufes['diagnostic'].replace(
    {'BCC': 'basal_cell_carcinoma', 'SEK': 'seborreic_keratosis', 'SCC': 'squamous_cell_carcinoma', 'NEV': 'nevus',
     'ACK': 'actinic_keratosis', 'MEL': 'melanoma'})

df_pad_ufes['im_dir'] = df_pad_ufes['img_id'].apply(lambda x: PAD_UFES_PATH + '/images/' + x)
df_pad_ufes['dataset'] = 'PAD_UFES'
display(df_pad_ufes)

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed,im_dir,dataset
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,False,False,False,False,False,False,PAT_1516_1765_530.png,False,datasets/PAD_UFES_20/images/PAT_1516_1765_530.png,PAD_UFES
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,True,True,False,True,True,True,PAT_46_881_939.png,True,datasets/PAD_UFES_20/images/PAT_46_881_939.png,PAD_UFES
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,True,False,False,False,False,False,PAT_1545_1867_547.png,False,datasets/PAD_UFES_20/images/PAT_1545_1867_547.png,PAD_UFES
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,True,False,False,False,False,False,PAT_1989_4061_934.png,False,datasets/PAD_UFES_20/images/PAT_1989_4061_934.png,PAD_UFES
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,True,True,False,False,True,True,PAT_684_1302_588.png,True,datasets/PAD_UFES_20/images/PAT_684_1302_588.png,PAD_UFES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,True,False,False,False,False,False,PAT_1708_3156_175.png,False,datasets/PAD_UFES_20/images/PAT_1708_3156_175.png,PAD_UFES
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,True,True,False,True,False,False,PAT_46_880_140.png,True,datasets/PAD_UFES_20/images/PAT_46_880_140.png,PAD_UFES
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,False,False,False,False,False,False,PAT_1343_1217_404.png,False,datasets/PAD_UFES_20/images/PAT_1343_1217_404.png,PAD_UFES
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,True,False,False,False,False,True,PAT_326_690_823.png,True,datasets/PAD_UFES_20/images/PAT_326_690_823.png,PAD_UFES


Ahora, solo quedaría añadir la información al csv global manteniendo el formato establecido

In [13]:
df_final_pad_ufes = df_pad_ufes[['img_id', 'im_dir', 'diagnostic', 'dataset']].to_numpy()
with open("preprocessedData.csv", "a", newline="") as archivo_csv:
    escritor_csv = csv.writer(archivo_csv)
    escritor_csv.writerows(df_final_pad_ufes)  # Escribe la información de los archivos

### PH2

 Este dataset recoge imágenes provenientes del Servicio de Dermatología del Hospital Pedro Hispano (Matosinhos, Portugal), que recoge pruebas cutáneas realizadas con el sistema Tuebinger Mole Analyzer, y un aumento de 20x. Son imágenes a color de triple canal con una resolución de 768x560 píxeles.

Esta base de datos de imágenes contiene un total de 200 imágenes dermatoscópicas de lesiones melanocíticas, incluidos 80 lunares comunes, 80 nevus atípicos y 40 melanomas. La base de datos PH² incluye anotaciones médicas de todas las imágenes: segmentación médica de la lesión, diagnóstico clínico e histológico, y evaluación de varios criterios dermatoscópicos. Entre ellos, encontramos distinción por colores; formación del tejido; puntos/glóbulos; rayas; áreas de regresión; velo azul blanquecino (pgimentación difusa).

#### Estructura de los datos
 
Este datset se organiza por carpetas codificadas, donde encontrarmos en su interior la imagen dermoscópica, la plantilla de segmentación y otras regiones de interés en directorios diferentes. En nuestro caso, seleccionaremos de momento únicamente las imágenes presentes en el directorio de imagen dermoscópica, que contiene la lesión en sí.

Se procede a la lectura del fichero csv que contiene los metadatos, y la adición de todas las entradas de archivos a la tabla común de todos los datasets.

In [14]:
# Lectura del csv dado
df_ph2 = pd.read_excel(PH2_PATH + '/PH2_dataset.xlsx', skiprows=range(0, 12))
display(df_ph2)

,Image Name,Histological Diagnosis,Common Nevus,Atypical Nevus,Melanoma,Asymmetry\n(0/1/2),Pigment Network\n(AT/T),Dots/Globules\n(A/AT/T),Streaks\n(A/P),Regression Areas\n(A/P),Blue-Whitish Veil\n(A/P),White,Red,Light-Brown,Dark-Brown,Blue-Gray,Black
0,IMD003,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,NaN,X,NaN,NaN
1,IMD009,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
2,IMD016,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,X,X,NaN,NaN
3,IMD022,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
4,IMD024,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,X,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IMD424,Melanoma,NaN,NaN,X,2,AT,AT,P,A,P,NaN,NaN,X,X,X,X
196,IMD425,Melanoma,NaN,NaN,X,2,AT,AT,A,P,P,NaN,NaN,X,X,X,X
197,IMD426,Melanoma,NaN,NaN,X,2,AT,A,A,P,A,NaN,NaN,X,X,NaN,NaN
198,IMD429,NaN,NaN,NaN,X,0,AT,A,P,A,P,NaN,NaN,X,X,X,X


Las columnas que contienen una x son aquellas que indican la clase del lunar que se está observando. Puede ser un lunar atípico, o un lunar o comun, o un melanoma. Codificaremos los tres tipos adecuadamente para construir un nuevo csv del que realizar la mezcla y separación train-test.

In [15]:
new_labels_ph2 = []
for index, row in df_ph2.iterrows():
    # Actualizar los valores de las columnas según alguna lógica
    if row['Common Nevus'] == 'X':
        new_labels_ph2.append("nevus")
    elif row['Atypical Nevus'] == 'X':
        new_labels_ph2.append("atypical nevus")
    elif row['Melanoma'] == 'X':
        new_labels_ph2.append("melanoma")
        
# Adjuntamos la nueva columna a las columnas a conservar
df_ph2["class"] = new_labels_ph2

Ahora, sólo debemos extraer las columnas deseadas e insertar los valores al csv global.

In [16]:

df_ph2['im_dir'] = df_ph2['Image Name'].apply(lambda x: PH2_PATH + '/PH2 Dataset images/' + x + f"{x}_Dermoscopic_Image/" + x + ".bmp")
df_ph2['dataset'] = 'PH2'
#display(df_ph2)

df_final_ph2 = df_ph2[['Image Name', 'im_dir', 'class', 'dataset']].to_numpy()

with open("preprocessedData.csv", "a", newline="") as archivo_csv:
    escritor_csv = csv.writer(archivo_csv)
    escritor_csv.writerows(df_final_ph2)  # Escribe la información de los archivos